# Shared Task
## Preparations

In [78]:
import pandas as pd
import spacy
import os
import csv
import nltk

nltk.download('wordnet')
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from nltk.corpus import wordnet as wn
from tqdm import tqdm_notebook as tqdm
from pprint import pprint
from spacy import displacy
from enum import Enum

nlp = spacy.load('en_core_web_lg')
pd.set_option('display.max_colwidth',-1)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\syim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**LOAD DATA**

In [82]:
folder = '../Documents/ELEXIS/codalab/public_dat/train'

#all_data = pd.read_csv('../Documents/ELEXIS/codalab/public_dat/train/english_kd.tsv',sep='\t', header=None)
all_data = pd.read_csv(folder + '/' + 'english_nuig.tsv', sep='\t', header=None)

           0          1                                                 2  \
0     squall  verb       blow in a squall                                   
1     squall  verb       make high-pitched, whiney noises                   
2     squall  verb       utter a sudden loud cry                            
3     tumid   adjective  abnormally distended especially by fluids or gas   
4     tumid   adjective  abnormally distended especially by fluids or gas   
...     ...         ...                                               ...   
8332  offer   verb       mount or put up                                    
8333  offer   verb       mount or put up                                    
8334  offer   verb       mount or put up                                    
8335  offer   verb       mount or put up                                    
8336  offer   verb       mount or put up                                    

                                                                           

**Define Feature Extraction related functions**

In [86]:
def add_column_names(df):
    column_names = ['word','pos','def1','def2','relation']
    df.columns = column_names
    
def load_data(file_path):
    loaded_data = pd.read_csv(file_path, sep='\t', header=None)
    add_column_names(loaded_data)
    
    return loaded_data

def load_training_data(folder):
    all_data = {}
    
    for filename in os.listdir(folder):
        if filename.endswith(".tsv"):
            all_data[filename.split('.')[0]] = load_data(folder + '/' + filename)
    
    return all_data


{'broader', 'narrower', 'none', 'related', 'exact'}


In [219]:
all_data = load_training_data(folder)
en_data = all_data['english_kd']
print(set(en_data['relation']))


{'broader', 'narrower', 'none', 'related', 'exact'}


**Add Text Classifier to the pipeline**

**Print only narrower relations**

In [220]:
#df['def1']=df['def1'].str.wrap(20)
is_narrower = en_data['relation']=='narrower'
is_not_none = en_data['relation']!='none'

print(en_data[is_not_none])

          word          pos  \
0    off        preposition   
3    off        preposition   
8    off        preposition   
13   off        preposition   
15   off        adverb        
..   ...           ...        
530  on         preposition   
551  on         preposition   
552  one        number        
553  one        number        
554  offspring  noun          

                                                    def1  \
0    away from and no longer touching                      
3    in a position away from                               
8    not inside a large vehicle used by the public         
13   not eating or taking                                  
15   away from a place                                     
..                 ...                                     
530  indicates sth or sb uses a type of food, fuel, etc.   
551  immediately following                                 
552  the number 1                                          
553  the number 1          

**Run Spacy NLP Pipeline**

In [227]:
def spacyDocForVec(vec):
    doc_list = []
    
    for doc in tqdm(vec):
        pr = nlp(doc)
        doc_list.append(pr)
    
    return doc_list


modDfObj = en_data.apply(lambda x: spacyDocForVec(x) if x.name in ['def1', 'def2'] else x)
modDfObj
#doc_list = spacyDocForVec(en_data['def1'])
#doc_list2 = spacyDocForVec(en_data['def2'])

,word,pos,def1,def2,relation
0,off,preposition,"(away, from, and, no, longer, touching)","(away, from, ;, down, from)",narrower
1,off,preposition,"(away, from, and, no, longer, touching)","(not, wanting, or, allowed, to, have, (, food, etc, ))",none
2,off,preposition,"(away, from, and, no, longer, touching)","(out, of, (, a, vehicle, ,, train, etc, ))",none
3,off,preposition,"(in, a, position, away, from)","(away, from, ;, down, from)",narrower
4,off,preposition,"(in, a, position, away, from)","(not, wanting, or, allowed, to, have, (, food, etc, ))",none
...,...,...,...,...,...
551,on,preposition,"(immediately, following)","(followed, by)",related
552,one,number,"(the, number, 1)","(the, number, or, figure, 1)",exact
553,one,number,"(the, number, 1)","(the, age, of, 1)",related
554,offspring,noun,"(a, person, 's, child, or, an, animal, 's, baby)","((, formal, ,, humorous, ), someone, 's, child, or, children)",broader


In [89]:
#df['def1_nlp']=doc_list
#df['def2_nlp']=doc_list2

In [252]:
modDfObj['def1'][0].text

'away from and no longer touching'

In [229]:
t1 = doc_list2[0]
for token in doc_list2[0]
    print(token.text, "| lemma:", token.lemma_, "| norm:" , token.norm_, "| pos:" ,token.pos_, "| tag:", token.tag_, "| dep:", token.dep_, "| sentiment:", token.sentiment)


SyntaxError: invalid syntax (<ipython-input-229-d82e7d6497eb>, line 2)

In [245]:
en_data
frame = pd.DataFrame({'doc1': doc_list, 'doc2': doc_list2})
en_data['doc1'] = frame['doc1']
en_data['doc1'] = frame['doc2]

0      away from and no longer touching    
1      away from and no longer touching    
2      away from and no longer touching    
3      in a position away from             
4      in a position away from             
                ...                        
551    immediately following               
552    the number 1                        
553    the number 1                        
554    a person's child or an animal's baby
555    a person's child or an animal's baby
Name: def1, Length: 556, dtype: object

In [ ]:
en_data

In [ ]:
def sentence2vec(row):
    return row['def1'].similarity(row['def2'])

    
def first_word_same(row):
     return (row['def1'].split(' ')[0].lower() == row['def2'].split(' ')[0].lower())

def difference_in_length(row):
    return abs(len(row['def1'].split(' ')) - len(row['def2'].split(' ')[0]))


features = pd.DataFrame()

similarities = []
first_word = []
length = []

features['similarities'] = modDfObj.apply(lambda row: sentence2vec(row), axis = 1)

#for i, row in en_data.iterrows():
#    similarities.append(sentence2Vec(row))
#    first_word.append(first_word_same(row))
#    length.append(difference_in_length(row)) 

#features['similarities'] = similarities
#features['first_word_same'] = first_word
#features['length'] = length
#Add Token Count using CountVectorizer

In [ ]:
features

In [ ]:
#en_data['similarities'] = similarities
labels = en_data['relation']
features['relation'] = en_data['relation']

features_nltk = []
for index, row in features.iterrows():
    features_nltk.append((row.similarities, row.relation))
    
features_nltk

**Split Dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

X_train_scaled = X_train.copy(deep = True)
X_test_scaled = X_test.copy(deep = True)

X_train_scaled['similarities'] = preprocessing.scale(X_train['similarities'])
X_train_scaled['length'] = preprocessing.scale(X_train['length'])
X_test_scaled['similarities'] = preprocessing.scale(X_test['similarities'])
X_test_scaled['length'] = preprocessing.scale(X_test['length'])

X_train_set = {}
X_train_set['unscaled'] = X_train
X_train_set['scaled'] = X_train_scaled

X_test_set = {}
X_test_set['unscaled'] = X_test
X_test_set['scaled'] = X_test_scaled


In [ ]:
def get_baseline(test_set):
    TP = 0
    for index in test_set.index:
        if test_set[index]=='none':
            TP+=1

    return float(TP/len(test_set))

models = {}
models['unscaled'] = [LR, SVM, RF]
models['scaled'] = [LR_scaled, SVM_scaled, RF_scaled]

def run_cv_with_dataset(model, trainset, y_train):
    scores = cross_val_score(model, trainset, y_train, cv = 5)        
    print('Cross validation scores for model' + model.__class__.__name__)
    scores
    print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

def cross_val_models(models, all_training_set, y_train):
    for estimator in models['unscaled']:
        run_cv_with_dataset(estimator, all_training_set['unscaled'], y_train)
    
    for estimator in models['scaled']:
        run_cv_with_dataset(estimator, all_training_set['scaled'], y_train)

def compare_on_testset(models, testset_x, testset_y, testset_x_scaled):
    print('Model Evaluation on Testset: ' + '\n')
    print('\t' + 'BASELINE: ' + str(get_baseline(testset_y)) + '\n')
    
    for estimator in models.scaled:
        estimator.predict(testset_x)
        print('\t' + estimator.__class__.__name__)
        score = estimator.score(testset_x, testset_y)
        print('\t\t' + "Accuracy: %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2) + '\n')
        
        estimator.predict(testset_x_scaled)
        score = estimator.score(testset_x_scaled, testset_y)
        print('\t\t' + "Accuracy for scaled featureset: %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2) + '\n')

def train_models(X_train, y_train, X_train_scaled):
    print('baseline: ', str(get_baseline(y_test)) + '\n')

    LR = LogisticRegression( solver = 'lbfgs', multi_class = 'multinomial').fit(X_train, y_train)
    LR_scaled = LogisticRegression( solver = 'lbfgs', multi_class = 'multinomial').fit(X_train_scaled, y_train)

    ## Linear kernal won't work very well, experiment with nonlinear ones.
    SVM = svm.LinearSVC(C=10.0).fit(X_train, y_train)
    SVM_scaled = svm.LinearSVC().fit(X_train_scaled, y_train)

    RF = RandomForestClassifier(max_depth = 5, random_state=0).fit(X_train, y_train)
    RF_scaled = RandomForestClassifier(max_depth = 5, random_state=0).fit(X_train_scaled, y_train)
    
    models = {}
    models['unscaled'] = [LR, SVM, RF]
    models['scaled'] = [LR_scaled, SVM_scaled, RF_scaled]
    
    return models

models = train_models(X_train, y_train, X_train_scaled)
cross_val_models(models, X_train_set, y_train)
#compare_on_testset(models, X_test, y_test, X_test_scaled)


In [246]:
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe(
        "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
    )
    nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe("textcat")
    
textcat.add_label('related')
textcat.add_label('exact')
textcat.add_label('broader')
textcat.add_label('narrower')
textcat.add_label('none')




,word,pos,def1,def2,relation,doc1,doc2
0,off,preposition,away from and no longer touching,away from; down from,narrower,"(blow, in, a, squall)","(to, cry, out, ;, to, scream, or, cry, violently, ,, as, a, woman, frightened, ,, or, a, child, in, anger, or, distress, ;, .)"
1,off,preposition,away from and no longer touching,not wanting or allowed to have (food etc),none,"(make, high, -, pitched, ,, whiney, noises)","(to, cry, out, ;, to, scream, or, cry, violently, ,, as, a, woman, frightened, ,, or, a, child, in, anger, or, distress, ;, .)"
2,off,preposition,away from and no longer touching,"out of (a vehicle, train etc)",none,"(utter, a, sudden, loud, cry)","(to, cry, out, ;, to, scream, or, cry, violently, ,, as, a, woman, frightened, ,, or, a, child, in, anger, or, distress, ;, .)"
3,off,preposition,in a position away from,away from; down from,narrower,"(abnormally, distended, especially, by, fluids, or, gas)","(swelled, ,, enlarged, ,, or, distended, ;)"
4,off,preposition,in a position away from,not wanting or allowed to have (food etc),none,"(abnormally, distended, especially, by, fluids, or, gas)","(rising, above, the, level, ;, protuberant, .)"
...,...,...,...,...,...,...,...
551,on,preposition,immediately following,followed by,related,"(the, act, of, rendering, a, person, legitimate)","(the, act, of, making, legitimate, .)"
552,one,number,the number 1,the number or figure 1,exact,"(the, act, of, rendering, a, person, legitimate)","(lawful, birth, .)"
553,one,number,the number 1,the age of 1,related,"(having, services, engaged, for, a, fee)","(performing, work, for, pay, ;, .)"
554,offspring,noun,a person's child or an animal's baby,"(formal, humorous) someone's child or children",broader,"(hired, for, the, exclusive, temporary, use, of, a, group, of, travelers)","(performing, work, for, pay, ;, .)"


In [256]:
def sentence2vec(row):
    return row['def1'].similarity(row['def2'])

    
def first_word_same(row):
     return (row['def1'].split(' ')[0].lower() == row['def2'].split(' ')[0].lower())

def difference_in_length(row):
    return abs(len(row['def1'].split(' ')) - len(row['def2'].split(' ')[0]))


features = pd.DataFrame()

similarities = []
first_word = []
length = []

features['similarities'] = modDfObj.apply(lambda row: sentence2vec(row), axis = 1)

#for i, row in en_data.iterrows():
#    similarities.append(sentence2Vec(row))
#    first_word.append(first_word_same(row))
#    length.append(difference_in_length(row)) 

#features['similarities'] = similarities
#features['first_word_same'] = first_word
#features['length'] = length
#Add Token Count using CountVectorizer

In [257]:
features

,similarities
0,0.856918
1,0.776428
2,0.732969
3,0.836521
4,0.689816
...,...
551,0.615553
552,0.915048
553,0.822070
554,0.850948


In [274]:
#en_data['similarities'] = similarities
labels = en_data['relation']
features['relation'] = en_data['relation']

features_nltk = []
for index, row in features.iterrows():
    features_nltk.append((row.similarities, row.relation))
    
features_nltk

[(0.8569175341853327, 'narrower'),
 (0.7764277625570569, 'none'),
 (0.7329691885840767, 'none'),
 (0.8365211865023756, 'narrower'),
 (0.6898163824998144, 'none'),
 (0.7367931309525856, 'none'),
 (0.6803962102027121, 'none'),
 (0.7521427806527426, 'none'),
 (0.7952153810359776, 'exact'),
 (0.6636760581748231, 'none'),
 (0.5763708126399653, 'none'),
 (0.6069563878788341, 'none'),
 (0.6514210433286518, 'none'),
 (0.8525577768853777, 'exact'),
 (0.7123831075808771, 'none'),
 (0.8482650807614045, 'narrower'),
 (0.6902407678790385, 'none'),
 (0.6923127331787525, 'none'),
 (0.5812013030610397, 'none'),
 (0.756551017992658, 'none'),
 (0.5780646498032915, 'none'),
 (0.7950366069506826, 'none'),
 (0.2533819159008091, 'none'),
 (0.735824786706821, 'none'),
 (0.662430446505536, 'none'),
 (0.6013527165121311, 'none'),
 (0.6239238959155543, 'none'),
 (0.708547657928058, 'none'),
 (0.566483874075168, 'none'),
 (0.676458026050009, 'broader'),
 (0.33422665314116606, 'none'),
 (0.7805555764860433, 'none

**Split Dataset**

In [201]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

X_train_scaled = X_train.copy(deep = True)
X_test_scaled = X_test.copy(deep = True)

X_train_scaled['similarities'] = preprocessing.scale(X_train['similarities'])
X_train_scaled['length'] = preprocessing.scale(X_train['length'])
X_test_scaled['similarities'] = preprocessing.scale(X_test['similarities'])
X_test_scaled['length'] = preprocessing.scale(X_test['length'])

X_train_set = {}
X_train_set['unscaled'] = X_train
X_train_set['scaled'] = X_train_scaled

X_test_set = {}
X_test_set['unscaled'] = X_test
X_test_set['scaled'] = X_test_scaled


In [208]:
def get_baseline(test_set):
    TP = 0
    for index in test_set.index:
        if test_set[index]=='none':
            TP+=1

    return float(TP/len(test_set))

models = {}
models['unscaled'] = [LR, SVM, RF]
models['scaled'] = [LR_scaled, SVM_scaled, RF_scaled]

def run_cv_with_dataset(model, trainset, y_train):
    scores = cross_val_score(model, trainset, y_train, cv = 5)        
    print('Cross validation scores for model' + model.__class__.__name__)
    scores
    print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

def cross_val_models(models, all_training_set, y_train):
    for estimator in models['unscaled']:
        run_cv_with_dataset(estimator, all_training_set['unscaled'], y_train)
    
    for estimator in models['scaled']:
        run_cv_with_dataset(estimator, all_training_set['scaled'], y_train)

def compare_on_testset(models, testset_x, testset_y, testset_x_scaled):
    print('Model Evaluation on Testset: ' + '\n')
    print('\t' + 'BASELINE: ' + str(get_baseline(testset_y)) + '\n')
    
    for estimator in models.scaled:
        estimator.predict(testset_x)
        print('\t' + estimator.__class__.__name__)
        score = estimator.score(testset_x, testset_y)
        print('\t\t' + "Accuracy: %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2) + '\n')
        
        estimator.predict(testset_x_scaled)
        score = estimator.score(testset_x_scaled, testset_y)
        print('\t\t' + "Accuracy for scaled featureset: %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2) + '\n')

def train_models(X_train, y_train, X_train_scaled):
    print('baseline: ', str(get_baseline(y_test)) + '\n')

    LR = LogisticRegression( solver = 'lbfgs', multi_class = 'multinomial').fit(X_train, y_train)
    LR_scaled = LogisticRegression( solver = 'lbfgs', multi_class = 'multinomial').fit(X_train_scaled, y_train)

    ## Linear kernal won't work very well, experiment with nonlinear ones.
    SVM = svm.LinearSVC(C=10.0).fit(X_train, y_train)
    SVM_scaled = svm.LinearSVC().fit(X_train_scaled, y_train)

    RF = RandomForestClassifier(max_depth = 5, random_state=0).fit(X_train, y_train)
    RF_scaled = RandomForestClassifier(max_depth = 5, random_state=0).fit(X_train_scaled, y_train)
    
    models = {}
    models['unscaled'] = [LR, SVM, RF]
    models['scaled'] = [LR_scaled, SVM_scaled, RF_scaled]
    
    return models

models = train_models(X_train, y_train, X_train_scaled)
cross_val_models(models, X_train_set, y_train)
#compare_on_testset(models, X_test, y_test, X_test_scaled)


baseline:  0.8561151079136691



C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\syim\AppData\Local\Continuum\an

Cross validation scores for modelLR scaled
Accuracy: 0.8585 (+/- 0.0020)


C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: C

Cross validation scores for modelLinearSVC
Accuracy: 0.8534 (+/- 0.0103)
Cross validation scores for modelRandomForestClassifier
Accuracy: 0.8625 (+/- 0.0087)
Cross validation scores for modelLR scaled
Accuracy: 0.8594 (+/- 0.0030)


C:\Users\syim\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Cross validation scores for modelLinearSVC
Accuracy: 0.8561 (+/- 0.0013)
Cross validation scores for modelRandomForestClassifier
Accuracy: 0.8625 (+/- 0.0087)


In [23]:
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe(
        "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
    )
    nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe("textcat")
    
textcat.add_label('related')
textcat.add_label('exact')
textcat.add_label('broader')
textcat.add_label('narrower')
textcat.add_label('none')




# TODO
## Experiment with NLP Pipeline

In [16]:
def lemmatizer(doc):
        # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
#nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
#nlp.add_pipe(remove_stopwords, name="stopwords", last=True)